In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib


def train_model(data_folder, model_name):
    X, y = [], []

    for filename in os.listdir(data_folder):
        if filename.endswith(".csv"):
            label = filename.replace(".csv", "")
            filepath = os.path.join(data_folder, filename)
            df = pd.read_csv(filepath)

            if df.empty:
                print(f"Warning: {filename} is empty, skipping")
                continue

            X.append(df.values)
            y.extend([label] * len(df))

    if not X:
        print(f"No data found in {data_folder}, skipping training for {model_name}")
        return

    X = np.vstack(X)
    y = np.array(y)

    print(f"\nStarting training for {model_name.upper()}")
    print(f"Total Samples: {len(y)}")
    print(f"Classes: {set(y)}")

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = RandomForestClassifier(n_estimators=150, random_state=42)
    model.fit(X_train_scaled, y_train)

    y_pred = model.predict(X_test_scaled)

    acc = accuracy_score(y_test, y_pred)
    print(f"\nAccuracy: {acc * 100:.2f}%\n")

    print("Classification Report:")
    print(classification_report(y_test, y_pred, digits=4))

    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    # Save model and scaler
    joblib.dump(model, f"{model_name}.pkl")
    joblib.dump(scaler, f"{model_name}_scaler.pkl")
    print(f"\nSaved files: {model_name}.pkl and {model_name}_scaler.pkl")


def main():
    train_model("dataset/face", "face_model")
    train_model("dataset/hand", "hand_model")


if __name__ == "__main__":
    main()
